### Hourly forecasting of energy meter readings on BDG2 dataset

- historical data = 1 week (168 data points)
- forecast horizon = 1 day (24 data points)

**Loading TimesFM Model**

In [1]:
import os
import glob
import time
from datetime import datetime
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import islice

from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from skforecast.ForecasterAutoreg import ForecasterAutoreg

import warnings
warnings.filterwarnings('ignore') 

In [2]:
# Data pipelining
def get_batched_data_fn(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      #examples["country"].append(country)
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      #examples["gen_forecast"].append(sub_df["gen_forecast"][start:context_end + horizon_len].tolist())
      #examples["week_day"].append(sub_df["week_day"][start:context_end + horizon_len].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end + horizon_len)].tolist())
      examples['inputs_ts'].append(sub_df.index[start:(context_end := start + context_len)])
      examples["outputs_ts"].append(sub_df.index[context_end:(context_end + horizon_len)])

    return examples

In [ ]:
# Benchmark
batch_size = 32
context_len = 168
horizon_len = 24

def process_building(df):
    building_name = df.columns[0]
    df.columns = ['y']
    input_data = get_batched_data_fn(df, batch_size=500)
    # print(input_data)
    
    windows_all = []
    counter = 1
    for inputs_ts, inputs, outputs_ts, outputs in zip(input_data['inputs_ts'], 
                                                      input_data['inputs'], 
                                                      input_data['outputs_ts'], 
                                                      input_data['outputs']):
        
        input_df = pd.DataFrame({'timestamp': inputs_ts, 
                                 'target': inputs})
        
        output_df = pd.DataFrame({'timestamp': outputs_ts, 
                                 'target': outputs})
        combined = pd.concat([input_df, output_df], axis=0)
        combined['item_id'] = str(building_name) + '_' + str(counter)
        combined['item_id_no'] = counter
        counter += 1
        windows_all.append(combined)
        
    windows_all_df = pd.concat(windows_all)
    windows_all_df.timestamp = pd.to_datetime(windows_all_df.timestamp)
    windows_all_df.set_index('timestamp', inplace=True)

    return windows_all_df

In [ ]:
def process_file(filename):
    df = pd.read_csv(filename)
    df = df.set_index(['timestamp'])
    df.index = pd.to_datetime(df.index)
    df['month'] = df.index.month
    training_set = df[df.month <= 6]
    test_set = df[df.month > 6]
    training_set = training_set.drop(columns='month')
    test_set = test_set.drop(columns='month')
    df = df.drop(columns='month')
    

    print(f'fine-tune set date range: {training_set.index[0]} {training_set.index[-1]}, '
      f'test set date range: {test_set.index[0]} {test_set.index[-1]}')
            

    if df.shape[1] < 2:
        return None
        
    print(datetime.now(), df.shape, flush=True)

    results_all = []
    c =1
    lag = 168 
    for building_name in df.columns:
        print(f'{datetime.now()} {c} / {len(df.columns)} {building_name}', flush=True)

        windowed_df_train = process_building(training_set[[building_name]])
        windowed_df_test = process_building(test_set[[building_name]])

        forecaster = ForecasterAutoreg(
                    regressor        = LinearRegression(),
                    lags             = 168
                )
        forecaster.fit(y= windowed_df_train['target'],
            )

        p = []
        for i in windowed_df_test.item_id_no.unique():
            # i -= 1           
            seq_ptr =lag + 24 * i
        
            df_test = windowed_df_test[windowed_df_test.item_id_no == i]
            last_window  = df_test.iloc[0:168]
            ground_truth = df_test.iloc[168:192]
        
            predictions = forecaster.predict(
                steps       = 24,
                last_window = last_window['target']
            )
            # p.append(predictions)
            res = ground_truth.copy()
            res = res[['target']]
            # print(res)
            res.columns = ['y_true']
            res = res.reset_index()
            res.insert(2, 'y_pred', predictions.reset_index()['pred'])
            res.set_index('timestamp', inplace=True)
            # res['y_pred'] = predictions
            p.append(res)
        res = pd.concat(p)
        res['building'] = building_name
        results_all.append(res)
        c+=1
        # if i == 2:
        #    break
        #break
        
    results_all_df = pd.concat(results_all)
    return results_all_df

In [5]:
files_list = glob.glob('/home/user/New_Buildings_Datasets/Mathura_and_Bareilly/dataverse_files/processed/Bareilly/*csv')

dataset = 'Bareilly-Linear'
os.makedirs(f'forecasts/{dataset}/', exist_ok = True)
os.makedirs(f'results/{dataset}/', exist_ok = True)

for filename in files_list:
    print(datetime.now(), filename)
    results = process_file(filename)
    if results is not None:
        results.to_csv(f'forecasts/{dataset}/{os.path.basename(filename)}')
    print('')

2024-11-13 13:14:12.831124 /home/user/New_Buildings_Datasets/Mathura_and_Bareilly/dataverse_files/processed/Bareilly/Bareilly_2021.csv
fine-tune set date range: 2021-01-01 00:00:00 2021-06-30 23:00:00, test set date range: 2021-07-01 00:00:00 2021-10-31 23:00:00
2024-11-13 13:14:12.866257 (7296, 38)
2024-11-13 13:14:12.866997 0 / 38 BR02
2024-11-13 13:14:13.744369 1 / 38 BR04
2024-11-13 13:14:14.654659 2 / 38 BR05
2024-11-13 13:14:15.475081 3 / 38 BR06
2024-11-13 13:14:16.380790 4 / 38 BR08
2024-11-13 13:14:17.228389 5 / 38 BR09
2024-11-13 13:14:18.148387 6 / 38 BR11
2024-11-13 13:14:18.990497 7 / 38 BR12
2024-11-13 13:14:19.887769 8 / 38 BR13
2024-11-13 13:14:20.780980 9 / 38 BR15
2024-11-13 13:14:21.600002 10 / 38 BR16
2024-11-13 13:14:22.485157 11 / 38 BR18
2024-11-13 13:14:23.304898 12 / 38 BR19
2024-11-13 13:14:24.206644 13 / 38 BR22
2024-11-13 13:14:25.035724 14 / 38 BR24
2024-11-13 13:14:25.907749 15 / 38 BR27
2024-11-13 13:14:26.727123 16 / 38 BR28
2024-11-13 13:14:27.595743 17

### Metrics

In [6]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import root_mean_squared_log_error
from permetrics.regression import RegressionMetric

dataset = 'Bareilly-Linear'
files_list = glob.glob(f'forecasts/{dataset}/*.csv')

metrics_all_files = []

for filename in files_list:
    print(filename)
    res = pd.read_csv(filename)
    metrics_all = []
    for (g, data) in res.groupby(['building']):
        data = data.dropna()
        data = data[data.y_pred >= 0]
        print(g[0]) 
        # print(data)
        if not data.empty:
            rmse= root_mean_squared_error(data.y_true, data.y_pred)
            mae= mean_absolute_error(data.y_true, data.y_pred)
            mape = mean_absolute_percentage_error(data.y_true, data.y_pred)
            mse= mean_squared_error(data.y_true, data.y_pred)
            msle= mean_squared_log_error(data.y_true, data.y_pred)
            rmsle= root_mean_squared_log_error(data.y_true, data.y_pred)
            nrmse = rmse / (data.y_true.mean()) 
    
            evaluator = RegressionMetric(data.y_true.to_list(), data.y_pred.to_list())
            nrmse_eve = evaluator.normalized_root_mean_square_error()
            evaluator = RegressionMetric(data.y_true.to_list(), data.y_pred.to_list())
            smape= evaluator.symmetric_mean_absolute_percentage_error()
        
            metrics = pd.DataFrame({'building_name': [g[0]], 
                               'mae': [mae],
                                'mape': [mape],
                               'mse': [mse], 'rmse': [rmse], 'msle': [msle], 'rmsle': [rmsle], 'nrmse' : [nrmse],
                                  'nrmse_eve':[nrmse_eve] , 'sMAPE' : [smape]})
            metrics_all.append(metrics)
        else:
            continue
    
    metrics_all_df = pd.concat(metrics_all)
    metrics_all_df.to_csv(f'results/{dataset}/{os.path.basename(filename)}')

    metrics_all_df['filename'] = os.path.basename(filename)
    metrics_all_files.append(metrics_all_df)

metrics_all_files_df = pd.concat(metrics_all_files)

forecasts/Bareilly-Linear/Bareilly_2021.csv
BR02
BR04
BR05
BR06
BR08
BR09
BR11
BR12
BR13
BR15
BR16
BR18
BR19
BR22
BR24
BR27
BR28
BR29
BR30
BR31
BR32
BR33
BR34
BR35
BR36
BR37
BR38
BR39
BR42
BR43
BR44
BR45
BR46
BR48
BR49
BR50
BR51
BR52
forecasts/Bareilly-Linear/Bareilly_2020.csv
BR02
BR03
BR04
BR05
BR06
BR07
BR08
BR09
BR10
BR11
BR12
BR13
BR14
BR15
BR16
BR17
BR18
BR19
BR20
BR22
BR23
BR24
BR26
BR27
BR28
BR29
BR30
BR31
BR32
BR33
BR34
BR35
BR36
BR37
BR38
BR39
BR42
BR43
BR44
BR45
BR46
BR48
BR49
BR50
BR51
BR52


In [7]:
metrics_all_files_df.to_csv(f'results/{dataset}/results_combined.csv')
metrics_all_files_df

,building_name,mae,mape,mse,rmse,msle,rmsle,nrmse,nrmse_eve,sMAPE,filename
0,BR04,0.256726,1.771933e+14,0.133643,0.365572,0.059803,0.244547,0.925921,1.113300,0.504743,Bareilly_2021.csv
0,BR05,0.109547,3.064832e+14,0.021054,0.145100,0.016853,0.129819,3.189394,1.708672,0.957248,Bareilly_2021.csv
0,BR06,0.387412,3.870723e+13,0.329226,0.573782,0.082027,0.286403,0.750404,1.241029,0.237787,Bareilly_2021.csv
0,BR11,0.094894,1.159648e+13,0.039347,0.198360,0.013615,0.116683,0.722700,1.094980,0.159239,Bareilly_2021.csv
0,BR13,0.121307,1.283129e+14,0.078018,0.279317,0.032127,0.179239,2.679092,1.300223,0.564757,Bareilly_2021.csv
...,...,...,...,...,...,...,...,...,...,...,...
0,BR48,0.035300,3.829195e+13,0.005660,0.075235,0.004347,0.065929,1.548455,0.902300,0.641005,Bareilly_2020.csv
0,BR49,0.311231,1.051232e+14,0.237545,0.487386,0.068275,0.261294,0.850526,0.970235,0.260715,Bareilly_2020.csv
0,BR50,0.124180,1.066671e+14,0.036181,0.190212,0.021684,0.147254,1.017366,1.065371,0.413218,Bareilly_2020.csv
0,BR51,0.488186,2.101831e+14,0.531194,0.728831,0.125865,0.354775,0.768014,0.974655,0.272853,Bareilly_2020.csv


In [8]:
metrics_all_files_df.describe()*100

,mae,mape,mse,rmse,msle,rmsle,nrmse,nrmse_eve,sMAPE
count,7900.000000,7.900000e+03,7900.000000,7900.000000,7900.000000,7900.000000,7800.000000,7900.000000,7900.000000
mean,18.604388,1.150371e+16,13.653295,28.310275,4.244011,17.207005,inf,263.927842,46.938229
std,16.674677,1.107248e+16,19.426218,23.897421,4.772888,11.400227,NaN,681.237087,25.445504
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,40.476822,68.981951,15.272369
25%,6.243981,4.469462e+15,1.286596,11.342719,0.871101,9.332443,75.856353,100.682041,27.708038
50%,12.479658,1.016531e+16,3.618072,19.021231,2.168378,14.725413,89.208939,114.594725,37.048180
75%,30.297505,1.545892e+16,21.481711,46.341980,6.870373,26.211269,132.474216,130.198578,59.099125
max,60.927463,7.211170e+16,68.408873,82.709656,17.902103,42.310877,inf,4596.380591,100.000000
